In [ ]:
# MOD 1: needed to append sys path to import holodeck
import sys
sys.path.append('C:/Users/emiga/OneDrive/Cal/GWs/code/holodeck')


# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

Nyquist Frequency Bins/Edges

In [ ]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
# print('fobs=', fobs)
# print('fobs_edges=',fobs_edges)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

SAM

In [ ]:
mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam = holo.sam.Semi_Analytic_Model(mtot=mtot, mrat=mrat, 
        redz=redz)
# gsmf=GSMF_Schechter, 
# gpf=GPF_Power_Law, 
# gmt=GMT_Power_Law, 
# mmbulge=relations.MMBulge_MM2013
   
# print('sam.mtot (M_sun)',sam.mtot/MSOL,
# '\nsam.mtot (g)', sam.mtot,
# '\nsam.mrat',sam.mrat,
# '\nsam.redz',sam.redz)
print('edges:', sam.edges)

gwb (internal)

In [ ]:
fobs_gw_edges = fobs_edges
hard = holo.hardening.Hard_GW
fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges)
# print('fobs_gw_cents', fobs_gw_cents)
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0
# print('fobs_orb_edges', fobs_orb_edges.shape)
# print('fobs_orb_cents', fobs_orb_cents.shape)


# dynamic_binary_number
# gets differential number of binaries per bin-vol
# per log frequency interval
edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
edges[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each
# bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_gw_edges))

# get GWB spectrum
hc = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False)
hc1 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=1)
        

In [ ]:
# learning
edges_test, dnum_test = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
edges_test[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each
# bin
number1_test = utils._integrate_grid_differential_number(edges_test, dnum_test, freq=False)
number2_test = number1_test * np.diff(np.log(fobs_gw_edges))

print(number1_test)
print('np.diff(log(fobs_gw_edges))\n', np.diff(np.log(fobs_gw_edges)))
print('number2\n', number2_test)

print our numbers

In [ ]:
for num in range(len(fobs_gw_cents)):
    print('\nfobs_gw_cents: %.2eHz' % fobs_gw_cents[num])
    print('fobs_orb_cents: %.2eHz ' % fobs_orb_cents[num])
    print('frequency bin %.2f nHz to %.2f nHz' % (edges[-1][num]*10**9, edges[-1][num+1]*10**9))
    np.set_printoptions(precision = 2)
    print('number =\n', number[:,:,:,num])

# GWs by Loops Dev

In [ ]:
foo = edges[-1] # observer frame orbital frequency
print('foo (obs frame orbital freq) =', foo)
df = np.diff(foo)
  # Q? What does np.diff do
  # A! n'th discrete difference along given axis
  #    just the differenc between neighboring values
print('df = diff foo =', df)
fc = kale.utils.midpoints(foo)
print('fc = obs frame orbital freq bin centers =', fc)

calculate GW strain

In [ ]:
# midpoints of mtot, mrat, and redz
mt = kale.utils.midpoints(edges[0])
mr = kale.utils.midpoints(edges[1])
rz = kale.utils.midpoints(edges[2])
np.set_printoptions(precision = 5)
print('mt = mtot midpoints =', mt)
print('mr = mrat midpoints =', mr)
print('rz = redz midpoints =',rz)

In [ ]:
mc = utils.chirp_mass_mtmr(mt[:,np.newaxis], 
                        mr[np.newaxis,:])
mc = mc[:, :, np.newaxis, np.newaxis]
print('mc = chirpmass = \n', mc)
dc = cosmo.comoving_distance(rz).cgs.value
dc = dc[np.newaxis, np.newaxis, :, np.newaxis]
print('dc = comoving distance =\n', dc)


In [ ]:
# convert from observer frame to rest frame
print('recall fc = obs frame orbital freq bin centers =', fc)
fr = utils.frst_from_fobs(fc[np.newaxis, :], rz[:, np.newaxis])
fr = fr[np.newaxis, np.newaxis, :, :]
print('fr = rest-frame orbital frequency centers =\n',
        fr)

strain amplitude of single source (separate for each bin), $h_s$
$$h_s = \frac{A M_c (2M_c f_{r,orb})^{2/3}}{ d_c}$$
where $A$ is a constant,
$$A = \frac{8 \times G^{5/3} \times \pi^{2/3}}{\sqrt(10) c^4}$$

characteristic strain for a single source (separate for each bin)
$$h_c^2 = h_s^2 \times \frac{f_\mathrm{obs,\ orb}}{{df_\mathrm{obs,\ orb}}} 
\\= h_s^2 \times \frac{\mathrm{obs\ frame\ orb\ freq\ center}}{\mathrm{freq\ bin\ width}}$$



* To realize: $h_c^2 = h_c^2 * \mathrm{poisson(number)}$ 
* Multiple realizations: same but with shape thing
* Unrealized: $h_c^2 = h_c^2 * number$ - adds up hc from all the binaries in the bin (not discrete)
\



Number $N = \int \frac{d^3N/dV}{dzd\log{M}dq d\ln{f_r}} d^3N dq dz d\log{M} dV$
If sum, 
$$h_c^2 = \sum_{M,q,z} h_{c,\mathrm{M,q,z}}^2$$

$$h_c = \sqrt{h_c}$$

All together: 
$$
h_c^2 = \sum_{M,q,z} \big(    h_s^2 \times \frac{f_\mathrm{obs,\ orb}}{{df_\mathrm{obs,\ orb}}} \big) \\

\sum_{M,q,z} \big(\frac{A M_c (2M_c f_{r,orb})^{2/3}}{ d_c} \frac{f_\mathrm{obs,\ orb}}{{df_\mathrm{obs,\ orb}}} \big)
$$


In [ ]:

# get gw strain from every individual source 
hs = utils.gw_strain_source(mc, dc, fr)
print('hs = single source strain =\n', hs, hs.shape)
print('recall edges =\n', edges)
print('2 total masses x 1 mass ratio x 3 redshifts x 5 frequencies')

In [ ]:
# characteristic strain calculated fromm GW strain
hc = (hs ** 2) * (fc / df)
print('hc = characteristic strain =\n', hc)

hc = (hs ** 2) * (fc / df)

$h_c = h_s^2 \times \frac{\mathrm{obs\ frame\ orb\ freq}}{\mathrm{freq\ bin\ width}}$

# GWs by Loops

In [ ]:
def gws_by_loops(edges, number, realize, print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    print_test : bool
        Whether or not to print variables as they are calculated, for dev purposes.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False: shape is (M-1, Q-1, Z-1, F-1)
        realize = R: shape is  (M-1, Q-1, Z-1, F-1, R)

    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    if(print_test):
        print('Observer frame frequency centers: ', fc)
        print('Mass edges: ', mt.shape, mt)
        print('Ratio edges:', mr.shape, mr)
        print('Redshift edges:', rz.shape, rz,'\n')

    # Chirp mass and comoving distance
    # mc = utils.chirp_mass_mtmr(mt[:, np.newaxis], mr[np.newaxis, :])
    # dc = holo.cosmo.comoving_distance(rz).cgs.value
    # if(print_test):
    #     print('Chirp mass', mc.shape, '\n', mc)
        # print('Comoving distance:', dc,'\n')

    # Convert freq bin centers to rest-frame
    # fr = utils.frst_from_fobs(fc[np.newaxis, :], rz[:, np.newaxis])
    # if(print_test):
    #     print('Rest frame frequency centers:', fr.shape, 
    #         '\n',fr,'\n')
    


    # make strain grid
    hc_grid = np.empty_like(number)

    # for r realizations, make a grid with shape 
    # m, q, z, f, r
    if(utils.isinteger(realize)):
        newshape = hc_grid.shape + (realize,)
        if(print_test):
            print('newshape:', newshape)
        realized_grid = np.empty(newshape)

    # get strain for each bin
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            for z_idx in range(len(rz)):
                cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                
                # print M, q, z, M_c, d_c
                if(print_test):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (cmass, cdist))

                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    
                    if(realize == False):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * number[m_idx, q_idx, z_idx, f_idx])
                    elif(realize == True):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                    elif(utils.isinteger(realize)):
                        for r_idx in range(realize):
                            realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                                np.sqrt(hc_dlnf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                        hc_grid = realized_grid                
                    else:
                        print("`realize` ({}) must be one of {{True, False, integer}}!"\
                            .format(realize))

                     # print fr, hs, hc^2/dlnf, number, hc
                    if(print_test):
                        print('\t\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (rfreq*10**9, hs_mqzf, hc_dlnf))
                        print('\t\tnumber: %.2e' % number[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_grid[m_idx, q_idx, z_idx, f_idx])
                    
            

    # if(realize == False):
    #     hc_grid *= number
    # elif(realize == True):
    #     hc_grid *= np.random.poisson(number)

    #not sure if this part will work
    # elif utils.isinteger(realize):
    #     try:
    #         realizations = np.empty(realize)
    #         for real in range(realize): 
    #             realizations[real]= gws_by_loops(edges, number, realize=True)
    #         hc_grid = realizations
    #     except:
    #         print("Something went wrong when you tried to do multiple realizations.")
    print('hc:', hc.shape)

    return hc_grid


    



In [ ]:
np.set_printoptions(precision = 4)
loop_hcF = gws_by_loops(edges, number, realize = False, print_test=True)
loop_hcT = gws_by_loops(edges, number, realize = True)
np.set_printoptions(precision = None)

In [ ]:
def gws_by_arrs(edges, number, realize, sum=False, print_test=False):


    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # ---- calculate GW strain ----
    mt = kale.utils.midpoints(edges[0])
    mr = kale.utils.midpoints(edges[1])
    rz = kale.utils.midpoints(edges[2])
    mc = utils.chirp_mass_mtmr(mt[:, np.newaxis], mr[np.newaxis, :])
    mc = mc[:, :, np.newaxis, np.newaxis]
    if(print_test):
        print('chirp mass: ', mc.shape, '\n', mc)
        print(mc[0,0,0,0])
        print(mc[1,0,0,0])
    dc = cosmo.comoving_distance(rz).cgs.value
    dc = dc[np.newaxis, np.newaxis, :, np.newaxis]

    if(print_test):
        print('Observer frame frequency centers: ', fc)
        print('Mass edges: ', mt.shape, mt)
        print('Ratio edges:', mr.shape, mr)
        print('Redshift edges:', rz.shape, rz,'\n')

    

    # convert from observer-frame to rest-frame; still using frequency-bin centers
    fr = utils.frst_from_fobs(fc[np.newaxis, :], rz[:, np.newaxis])
    fr = fr[np.newaxis, np.newaxis, :, :]

    hs = utils.gw_strain_source(mc, dc, fr)
    hc = (hs ** 2) * (fc / df)
    print('hs:', hs.shape,'\n',hs)
    print('hc^2/dlnf:', hc.shape,'\n',hc)
    # print('mt', len(mt))
    # print('dc', len(dc), dc.shape)
    # print chirp mass and comoving distance
    if(print_test):
        for m_idx in range(len(mt)):
            for q_idx in range(len(mr)):
                for z_idx in range(len(rz)):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    # print('m_idx = %d, q_idx = %d, z_idx=%d' % (m_idx, q_idx, z_idx))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (mc[m_idx, q_idx, 0,0], dc[0, 0, z_idx,0]))
                    for f_idx in range(len(fc)):
                        print('\t\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (fr[0, 0, 0, f_idx]*10**9, 
                            hs[m_idx, q_idx, z_idx, f_idx], hc[m_idx, q_idx, z_idx, f_idx]))  
                        print('\t\tnumber: %.2e' % number[m_idx, q_idx, z_idx, f_idx])
    # if(print_test):
    #     for m_idx in range(len(mt)):
    #         for q_idx in range(len(mr)):
    #             for z_idx in range(len(rz)):


    if realize is True:
        hc = hc * np.random.poisson(number)
    elif realize in [None, False]:
        # print('err here')
        # print('hc:\n', hc, '\nhc shape:', hc.shape)
        # print('number:\n', number, '\nnumber shape', number.shape)
        hc = hc * number
    elif utils.isinteger(realize):
        shape = number.shape + (realize,)
        try:
            hc = hc[..., np.newaxis] * np.random.poisson(number[..., np.newaxis], size=shape)
        except ValueError as err:
            log.error(str(err))
            print(f"{utils.stats(number)=}")
            print(f"{number.max()=:.8e}")
            print(f"{number.dtype=}")
            raise
    else:
        err = "`realize` ({}) must be one of {{True, False, integer}}!".format(realize)
        log.error(err)
        raise ValueError(err)

    # Sum over M, Q, Z bins
    # (M-1, Q-1, Z-1, F-1 [, R]) ==> (F-1, [, R])
    if sum:
        hc = np.sum(hc, axis=(0, 1, 2))

    # convert from hc^2 to hc
    hc = np.sqrt(hc)
    print('hc shape:', hc.shape)
    return hc

In [ ]:
np.set_printoptions(precision = 4)
print('\n-----------by loops--------')
hc_loopF = gws_by_loops(edges, number, realize = False, print_test=True)
print('\n-----------by grid arrays--------')
hc_arrF = gws_by_arrs(edges, number, realize = False, print_test=True)

# hc_arrT = gws_by_arrs(edgeas, number, realize = True)
np.set_printoptions(precision = None)

In [ ]:
# print(arr_hcF)
# print(loop_hcF)
print(hc_samF == hc_arrF)

# Test Cases

In [ ]:
# get GWB spectrum
hc_samF = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False, sum=False)
hc_samT = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=True, sum=False)
hc_sam1 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=1, sum=False)
hc_sam3 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=3, sum=False)

hc_loopF = gws_by_loops(edges, number, realize=False)    
hc_loopT = gws_by_loops(edges, number, realize=True)
hc_loop1 = gws_by_loops(edges, number, realize=1)   
hc_loop3 = gws_by_loops(edges, number, realize=3)   
        

In [ ]:
print(hc_loop3.shape)

In [ ]:
print(hc_sam3.shape)

In [ ]:
print(hc_loop3 - hc_sam3)

In [ ]:
print(hc_loopF)